In [660]:
import pandas as pd;
import numpy as np;


In [661]:
iris_Data = pd.read_csv('iris.csv');
iwd = iris_Data.copy();

In [695]:
iwd.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [663]:
col = iwd.columns

In [702]:
#Create a class to hold upperbound, lowerbound and chi values.
class chi:  
    def __init__(self, lower, upper,value):  
        self.lower = lower  
        self.upper = upper
        self.value = value

def ComputeChi(d):
    if d.shape[0] ==1 :
        return 0;
    temp =d.copy();  
    r = temp.sum(axis=0);
    c = temp.sum(axis=1);
    c = c/c.sum(axis=0);
    e = pd.DataFrame(np.zeros(temp.shape)); ##(c*temp/(c.sum()));
    for i in range(len(temp.columns)):
        ## calculate expected value.
        e.iloc[0,i] = c.iloc[0]*(temp.iloc[:,i].sum())/c.sum();
        e.iloc[1,i] = c.iloc[1]*(temp.iloc[:,i].sum())/c.sum();
        ## Calculate chi in the same array.
        e.iloc[0,i] = (temp.iloc[0,i]-e.iloc[0,i])**2/e.iloc[0,i]
        e.iloc[1,i] = (temp.iloc[1,i]-e.iloc[1,i])**2/e.iloc[1,i]
   
    return e.sum().sum();




def Chimerge(data):
    data =data.sort_values(by=[data.columns[0]]);
    d2 = pd.crosstab(data.iloc[:,0],data.iloc[:,1]);   
    intervals =data.iloc[:,0].unique(); 
    chilist = [];
    
    for i in range(len(d2.index)):
        y = chi(d2.index[i], d2.index[i],100000);
        chilist.append(y);
    
    while (len(chilist) > 6):    
        newlist =[];
        for i in range(len(chilist)-1): 
            newlist.append(chi(chilist[i].lower,chilist[i+1].upper, ComputeChi(d2.ix[chilist[i].lower:chilist[i].upper,:]))); 
         
        v = min(newlist, key=lambda x: x.value);
        for c in chilist:
            if c.lower == v.lower or c.upper == v.upper:
                del(c);
                
                
        chilist = [item for item in chilist if ( item.lower != v.lower  and item.upper != v.upper) ]
        
        chilist.append(v);
        chilist.sort(key=lambda x: x.lower);
       
    
    for c in chilist:
        print(c.lower, c.upper);

In [703]:
for i in range(1,len(col)-1):
    print(col[i]);
    print("=======================")
    Chimerge(iwd[[col[i],col[-1]]])

SepalLengthCm


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


4.3 4.9 0.0
5.0 5.5 0.27999999999999997
5.6 5.7 0
5.8 5.9 0
6.0 6.6 0.08888888888888888
6.7 7.9 0.02222222222222222
SepalWidthCm
2.0 2.4 0.4444444444444444
2.5 3.0 0.2
3.1 3.4 0.07122507122507135
3.5 3.6 0
3.7 3.8 0
3.9 4.4 0.0
PetalLengthCm
1.0 3.0 9.860761315262645e-32
3.3 4.5 1.972152263052529e-31
4.6 4.8 0.0
4.9 5.0 0
5.1 5.2 0
5.3 6.9 1.972152263052529e-31
PetalWidthCm
0.1 0.3 0.0
0.4 1.0 0.0
1.1 1.4 0.0
1.5 1.6 0
1.7 1.8 0
1.9 2.5 2.892489985810376e-31
